Importing libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kagglehub
import os
from kagglehub import KaggleDatasetAdapter

# Importing the dataset

### Downloading the Kaggle dataset

In [2]:
# Download latest version
path = kagglehub.dataset_download("vishalsubbiah/pokemon-images-and-types")

print("Path to dataset files:", path)

print(os.listdir(path))

100%|██████████| 3.68M/3.68M [00:00<00:00, 95.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/vishalsubbiah/pokemon-images-and-types/versions/4
['pokemon.csv', 'images']


In [3]:
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/vishalsubbiah/pokemon-images-and-types/versions/4


###Loading the dataset

In [4]:
# Set the path to the file you'd like to load
file_path = "pokemon.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "vishalsubbiah/pokemon-images-and-types",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

<ipython-input-4-701a89f64b34>:5: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


Download already complete (17053 bytes).
First 5 records:          Name  Type1   Type2   Evolution
0   bulbasaur  Grass  Poison     ivysaur
1     ivysaur  Grass  Poison    venusaur
2    venusaur  Grass  Poison         NaN
3  charmander   Fire     NaN  charmeleon
4  charmeleon   Fire     NaN   charizard


In [5]:
#combining the types
Type1 = set(df.loc[:,'Type1'].dropna())
Type2 = set(df.loc[:,'Type2'].dropna())
Type = Type1.union(Type2)
print(len(Type))

18


In [6]:
print(len(Type))

18


In [25]:
#remove directory and all files in it
shutil.rmtree('/content/Dataset/training_set/')
shutil.rmtree('/content/Dataset/test_set/')

# Creating the Directory Structure

In [7]:
import shutil

src = '/content'
img_dataset = f'{src}/drive/MyDrive/Pokemon_images'
#train_dst = f'{src}/training_set'
#test_dst = f'{src}/test_set'
Name = df.loc[:,'Name']

os.makedirs(img_dataset, exist_ok=True)
#os.mkdir(test_dst)

#making the class folders in both datasets
for x in Type:
  os.makedirs(f'{img_dataset}/{x}', exist_ok=True)
  #os.makedirs(f'{test_dst}/{x}', exist_ok=True)
  counter = 0

  for y in range(0,len(Name)):

    if df.loc[y,'Type1'] == x or df.loc[y,'Type2'] == x :
      Name_img = df.loc[y,'Name']
      shutil.copy(f'{src}/Dataset/{Name_img}.png',f'{img_dataset}/{x}')

      #creating train- test split manually
      #(not required as the utils.image_dataset_from does it)
      #if counter%10 == 0:
      #  shutil.copy(f'{src}/{Name_img}.png',f'{test_dst}/{x}')
      #else:
      #  shutil.copy(f'{src}/{Name_img}.png',f'{train_dst}/{x}')
      #counter = counter + 1


FileNotFoundError: [Errno 2] No such file or directory: '/content/Dataset/pikachu.png'

In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/Pokemon_images')

#Image augmentation

In [17]:
imge_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.Rescaling(1./255)
    #tf.keras.layers.RandomTranslation(0.2,0.2),
    #tf.keras.layers.RandomBrightness(0.2),
    #tf.keras.layers.RandomCrop(64,64)
])

##Loading the Dataset

In [16]:
input_dataset = tf.keras.utils.image_dataset_from_directory(
    img_dataset,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(64, 64),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None)

Found 1214 files belonging to 18 classes.


##Model 1

In [20]:
input = tf.keras.Input(shape=(64, 64, 3))

x = imge_augmentation(input)

x = tf.keras.layers.Conv2D(64, 3, strides = 1, activation='relu', input_shape=(64,64))(x)
x = tf.keras.layers.MaxPool2D(pool_size = (2,2), padding='valid', strides = 1)(x)

x = tf.keras.layers.Conv2D(64, 3, strides = 2, activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size = (2,2), padding='valid', strides = 1)(x)

x = tf.keras.layers.Conv2D(64, 3, strides = 2, activation='relu')(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(128, activation='relu')(x)

x = tf.keras.layers.Dense(128, activation='relu')(x)

output = tf.keras.layers.Dense(18, activation='softmax')(x)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


##Model 1 performance

In [21]:
model = tf.keras.models.Model(inputs=input, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(input_dataset, epochs = 50)

Epoch 1/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 185ms/step - accuracy: 0.0847 - loss: 2.8689
Epoch 2/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 170ms/step - accuracy: 0.1131 - loss: 2.8218
Epoch 3/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 11s 177ms/step - accuracy: 0.1374 - loss: 2.7652
Epoch 4/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 11s 198ms/step - accuracy: 0.1578 - loss: 2.7389
Epoch 5/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 11s 213ms/step - accuracy: 0.1507 - loss: 2.7096
Epoch 6/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 208ms/step - accuracy: 0.1620 - loss: 2.6948
Epoch 7/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 189ms/step - accuracy: 0.1401 - loss: 2.6868
Epoch 8/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 11s 203ms/step - accuracy: 0.1627 - loss: 2.6612
Epoch 9/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 214ms/step - accuracy: 0.1775 - loss: 2.6610
Epoch 10/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 217ms/step - accuracy: 0.1938 - loss: 2.5878
Epoch 11/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 170ms/step - accuracy: 0.2223 - loss: 2.5516
Epoch 12/100
38/38 ━━━━━━━━━━━━━━

##Model 2

In [18]:
input = tf.keras.Input(shape=(64, 64, 3))

x = imge_augmentation(input)

x = tf.keras.layers.Conv2D(12, 3, strides = 2, activation='relu', input_shape=(64,64))(x)
#x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size = (2,2), padding='valid', strides = 2)(x)

#x = tf.keras.layers.Conv2D(64, 3, strides = 2, activation='relu')(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.MaxPool2D(pool_size = (2,2), padding='valid', strides = 1)(x)

#x = tf.keras.layers.Conv2D(64, 3, strides = 2, activation='relu')(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.MaxPool2D(pool_size = (2,2), padding='valid', strides = 1)(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(32, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.2)(x)

#x = tf.keras.layers.Dense(128, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.2)(x)

output = tf.keras.layers.Dense(18, activation='softmax')(x)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


##Model 2 performance

In [19]:
model2 = tf.keras.models.Model(inputs=input, outputs=output)
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.fit(input_dataset, epochs = 10)

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 31s 760ms/step - accuracy: 0.0724 - loss: 2.8817
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 18s 151ms/step - accuracy: 0.1162 - loss: 2.8239
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 123ms/step - accuracy: 0.1327 - loss: 2.7888
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.1397 - loss: 2.7647
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 163ms/step - accuracy: 0.1406 - loss: 2.7651
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - accuracy: 0.1770 - loss: 2.7069
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 117ms/step - accuracy: 0.1713 - loss: 2.6903
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 153ms/step - accuracy: 0.1678 - loss: 2.6925
Epoch 9/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - accuracy: 0.1938 - loss: 2.6506
Epoch 10/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 131ms/step - accuracy: 0.1967 - loss: 2.6389
